In [ ]:
from ultralytics import YOLO
import os
import torch
import csv
from tqdm import tqdm

In [ ]:
model = YOLO('yolov8m-pose.pt')

In [ ]:
frame_path = 
csv_path = 

In [ ]:
def scale(x_list, y_list):
    y_min = min(y_list)
    y_max = max(y_list)

    multiplier = 320 / (y_max - y_min)

    x_list = [value + (abs(value - 320) * multiplier) for value in x_list]
    y_list = [value + (abs(value - 180) * multiplier) for value in y_list]

    return x_list, y_list

In [ ]:
def preprocess_keypoints(results):
    conf_list = results[0].keypoints.conf[0]
    conf_list = conf_list.tolist()
    
    x_list = [value[0] for value in results[0].keypoints.xy[0]]
    y_list = [value[1] for value in results[0].keypoints.xy[0]]

    try:
        x_list = [value[0] for value in results[0].keypoints.xy[0]]
        y_list = [value[1] for value in results[0].keypoints.xy[0]]

        zero_x_indices = [i for i, x in enumerate(x_list) if x == 0.0]
        zero_y_indices = [i for i, y in enumerate(y_list) if y == 0.0]
                
        temp_x_list = [x for x in x_list if x != 0.0]
        temp_y_list = [y for y in y_list if y != 0.0]

        x_min = min(temp_x_list)
        y_min = min(temp_y_list)
        x_max = max(temp_x_list)
        y_max = max(temp_y_list)

        norm_x = (x_min + x_max)/2
        norm_y = (y_min + y_max)/2

        x_list  = [item + (320 - norm_x) for item in temp_x_list]
        y_list = [item + (180 - norm_y) for item in temp_y_list]
        
        min_y = min(y_list).item()
        max_y = max(y_list).item()

        scaled_y_list = [(y - min_y) / (max_y - min_y) * 360 for y in y_list]
        scaled_x_list = [(x - min_y) / (max_y - min_y) * 360 for x in x_list]

        x_min = min(scaled_x_list)
        x_max = max(scaled_x_list)

        norm_x = (x_min + x_max)/2
        scaled_x_list  = [item + (320 - norm_x) for item in scaled_x_list]

        for i in zero_x_indices:
            scaled_x_list.insert(i, 0.0)
            conf_list[i] = 0.0
        
        for i in zero_y_indices:
            scaled_y_list.insert(i, 0.0)
            conf_list[i] = 0.0

        scaled_x_list = [float(element.item()) if torch.is_tensor(element) else float(element) for element in scaled_x_list]
        scaled_y_list = [float(element.item()) if torch.is_tensor(element) else float(element) for element in scaled_y_list]

        final_list = torch.tensor(list(zip(scaled_x_list, scaled_y_list)))
        return final_list
    
    except:
        return torch.tensor(list(zip(x_list, y_list)))

In [ ]:
def keypoint_generation(frame_path):
    keypoint_list = list()

    for severity in os.listdir(frame_path):
        for video in tqdm(os.listdir(os.path.join(frame_path, severity))):
            for orientation in os.listdir(os.path.join(frame_path, severity, video)):
                dict = {
                    'severity': severity,
                    'video': video,
                    'orientation': orientation,
                    'left-shoulder': [],
                    'right-shoulder': [],
                    'left-elbow': [],
                    'right-elbow': [],
                    'left-wrist': [],
                    'right-wrist': [],
                    'left-hip': [],
                    'right-hip': [],
                    'left-knee': [],
                    'right-knee': [],
                    'left-ankle': [],
                    'right-ankle': [],
                }
                if len(os.listdir(os.path.join(frame_path, severity, video, orientation))) > 0:
                    for image in os.listdir(os.path.join(frame_path, severity, video, orientation)):
                        results = model(os.path.join(frame_path, severity, video, orientation, image), verbose=False)
                        if len(results[0].keypoints.xy[0]) != 0:
                            keypoints = preprocess_keypoints(results)
                            if len(keypoints) != 0:
                                keypoints = keypoints.clone()
                                keypoints[keypoints == 0.0] = float('nan')

                                dict['left-shoulder'].append(keypoints[5])
                                dict['right-shoulder'].append(keypoints[6])
                                dict['left-elbow'].append(keypoints[7])
                                dict['right-elbow'].append(keypoints[8])
                                dict['left-wrist'].append(keypoints[9])
                                dict['right-wrist'].append(keypoints[10])
                                dict['left-hip'].append(keypoints[11])
                                dict['right-hip'].append(keypoints[12])
                                dict['left-knee'].append(keypoints[13])
                                dict['right-knee'].append(keypoints[14])
                                dict['left-ankle'].append(keypoints[15])
                                dict['right-ankle'].append(keypoints[16])
        
                    keypoint_list.append(dict)
                
    return keypoint_list

In [ ]:
def csv_keypoints(frame_path, csv_path):
    
    csv_file = open(csv_path, 'w', newline='')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow([
                        'Severity', 
                        'Video', 
                        'Orientation', 
                        'Frame', 
                        'Left-Shoulder', 
                        'Right-Shoulder', 
                        'Left-Elbow', 
                        'Right-Elbow', 
                        'Left-Wrist', 
                        'Right-Wrist', 
                        'Left-Hip', 
                        'Right-Hip', 
                        'Left-Knee', 
                        'Right-Knee', 
                        'Left-Ankle', 
                        'Right-Ankle'])

    for severity in os.listdir(frame_path):
        print(severity)
        for video in tqdm(os.listdir(os.path.join(frame_path, severity))):
            for orientation in os.listdir(os.path.join(frame_path, severity, video)):
                if len(os.listdir(os.path.join(frame_path, severity, video, orientation))) > 0:
                    for image in os.listdir(os.path.join(frame_path, severity, video, orientation)):
                        results = model(os.path.join(frame_path, severity, video, orientation, image), verbose=False)
                        if len(results[0].keypoints.xy[0]) != 0:
                            keypoints = preprocess_keypoints(results)
                            if len(keypoints) != 0:
                                keypoints = keypoints.clone()
                                keypoints[keypoints == 0.0] = float('nan')

                                row = [
                                    severity, 
                                    video, 
                                    orientation, 
                                    image, 
                                    keypoints[5], 
                                    keypoints[6], 
                                    keypoints[7], 
                                    keypoints[8], 
                                    keypoints[9], 
                                    keypoints[10], 
                                    keypoints[11], 
                                    keypoints[12], 
                                    keypoints[13], 
                                    keypoints[14], 
                                    keypoints[15], 
                                    keypoints[16]
                                ]

                        csv_writer.writerow(row)

    csv_file.close()

    return keypoint_list

In [ ]:
csv_keypoints(frame_path, csv_path)

In [ ]:
keypoint_list = keypoint_generation(frame_path)

In [ ]:
def mean_deviation(data):
    n = len(data)
    if n > 0:
        mean = sum(data) / n
        deviations = [abs(x - mean) for x in data]
        mean_dev = sum(deviations) / n
        return mean_dev
    else: 
        return torch.tensor([float('nan'), float('nan')], dtype=torch.float)

In [ ]:
def csv_write(keypoint_list, csv_path):

    csv_file = open(csv_path, 'w', newline='')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow([
                        'Severity', 
                        'Video', 
                        'Orientation', 
                        'Frame', 
                        'Left-Shoulder', 
                        'Right-Shoulder', 
                        'Left-Elbow', 
                        'Right-Elbow', 
                        'Left-Wrist', 
                        'Right-Wrist', 
                        'Left-Hip', 
                        'Right-Hip', 
                        'Left-Knee', 
                        'Right-Knee', 
                        'Left-Ankle', 
                        'Right-Ankle'])

    counter = 0

    for item in keypoint_list:
        row = [
            item['severity'],
            item['video'],
            item['orientation'],
            mean_deviation(item['left-shoulder']),
            mean_deviation(item['right-shoulder']),
            mean_deviation(item['left-elbow']),
            mean_deviation(item['right-elbow']),
            mean_deviation(item['left-wrist']),
            mean_deviation(item['right-wrist']),
            mean_deviation(item['left-hip']),
            mean_deviation(item['right-hip']),
            mean_deviation(item['left-knee']),
            mean_deviation(item['right-knee']),
            mean_deviation(item['left-ankle']),
            mean_deviation(item['right-ankle'])
        ]
        
        csv_writer.writerow(row)

        counter += 1
    print(counter)

In [ ]:
csv_write(keypoint_list, csv_path)

Keypoint List Structure

keypoint_list = [
    {
        'severity': severity_1, 
        'video': video_1,
        'orientation': orientation_1,
        'keypoint_1': [coord1, coord2, coord3, ... coordn],
        'keypoint_2': [coord1, coord2, coord3, ... coordn],
        ...
        'keypoint_12': [coord1, coord2, coord3, ... coordn]
    }, 
    {
        'severity': severity_2, 
        'video': video_2,
        'orientation': orientation_2,
        'keypoint_1': [coord1, coord2, coord3, ... coord12],
        'keypoint_2': [coord1, coord2, coord3, ... coord12],
        ...
        'keypoint_12': [coord1, coord2, coord3, ... coord12]
    },
    {
        'severity': severity_i, 
        'video': video_i,
        'orientation': orientation_i,
        'keypoint_1': [coord1, coord2, coord3, ... coord12],
        'keypoint_2': [coord1, coord2, coord3, ... coord12],
        ...
        'keypoint_12': [coord1, coord2, coord3, ... coord12]
    }
]